In [10]:
# -*- coding: utf-8 -*-
"""
Transport Suite for Jupyter Notebook:
NWC / LCM / VAM + optional Stepping-Stone
Interactive, step-by-step solution.
"""

from typing import List, Tuple, Set
import collections

TOL = 1e-12

# -------------------- Edit your data here --------------------
SUPPLY = [400, 500]
DEMAND = [300, 200, 400]
COST = [
    [600, 800, 700],
    [400, 900, 600],
]

# If you want to maximize, negate costs:
COST_NEG = [[-p for p in row] for row in COST]
# -------------------------------------------------------------

# -------------------- Utilities --------------------
def balance_transport(supply: List[float], demand: List[float], cost: List[List[float]]):
    """Balance problem by adding a dummy row/col if needed (cost=0)."""
    S, D = sum(supply), sum(demand)
    m, n = len(supply), len(demand)
    C = [row[:] for row in cost]
    added = {"dummy_row": False, "dummy_col": False}
    if abs(S - D) < 1e-9:
        return supply[:], demand[:], C, added
    if S < D:
        C.append([0.0] * n)
        supply = supply[:] + [D - S]
        added["dummy_row"] = True
    else:
        for r in C:
            r.append(0.0)
        demand = demand[:] + [S - D]
        added["dummy_col"] = True
    return supply, demand, C, added

def total_cost(plan, cost) -> float:
    m, n = len(plan), len(plan[0])
    return sum(plan[i][j] * cost[i][j] for i in range(m) for j in range(n))

def print_transport_table(plan: List[List[float]], cost: List[List[float]], title="Plan (qty@unit_cost)"):
    m, n = len(plan), len(plan[0])
    colw = 18
    sep = " | "
    print("\n" + title)
    print("-" * (colw * (n + 1) + len(sep) * n))
    header = ["Src/Dest".ljust(colw)] + [f"Dest{j+1}".center(colw) for j in range(n)]
    print(sep.join(header))
    print("-" * (colw * (n + 1) + len(sep) * n))
    for i in range(m):
        row = [f"Src{i+1}".ljust(colw)]
        for j in range(n):
            cell = f"{plan[i][j]:.4f}@{cost[i][j]:.2f}" if plan[i][j] != 0 else "".center(colw)
            row.append(cell.center(colw))
        print(sep.join(row))
    print("-" * (colw * (n + 1) + len(sep) * n))

# -------------------- Initial Methods --------------------
def northwest_corner(supply: List[float], demand: List[float], cost: List[List[float]], verbose=True):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0 for _ in range(n)] for _ in range(m)]
    if verbose:
        print("=== Northwest Corner Method ===")
        print("Balanced problem:")
        print(f"  supply = {supply}")
        print(f"  demand = {demand}")
        print(f"  added = {added}\n")
    i = j = 0
    step = 0
    while i < m and j < n:
        step += 1
        alloc = min(supply[i], demand[j])
        plan[i][j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc
        if verbose:
            print(f"Step {step}: allocate {alloc:.4f} at cell ({i},{j})  cost={cost[i][j]:.4f}")
            print(f"  Remaining: supply[{i}]={supply[i]:.4f}, demand[{j}]={demand[j]:.4f}\n")
        if abs(supply[i]) < TOL and abs(demand[j]) < TOL:
            i += 1; j += 1
        elif abs(supply[i]) < TOL:
            i += 1
        elif abs(demand[j]) < TOL:
            j += 1
    return plan, cost, added

def least_cost_method(supply: List[float], demand: List[float], cost: List[List[float]], verbose=True):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0 for _ in range(n)] for _ in range(m)]
    active_rows = {i for i in range(m) if supply[i] > 0}
    active_cols = {j for j in range(n) if demand[j] > 0}
    if verbose:
        print("=== Least Cost Method ===")
        print("Balanced problem:")
        print(f"  supply = {supply}")
        print(f"  demand = {demand}")
        print(f"  added = {added}\n")
    step = 0
    while active_rows and active_cols:
        step += 1
        best = None
        for i in active_rows:
            for j in active_cols:
                cand = (cost[i][j], i, j)
                if best is None or cand < best:
                    best = cand
        cmin, i, j = best
        alloc = min(supply[i], demand[j])
        plan[i][j] += alloc
        supply[i] -= alloc
        demand[j] -= alloc
        if verbose:
            print(f"Step {step}: choose cheapest cell ({i},{j}) with cost={cmin:.4f}")
            print(f"  Allocate {alloc:.4f}")
            print(f"  Remaining: supply[{i}]={supply[i]:.4f}, demand[{j}]={demand[j]:.4f}\n")
        if abs(supply[i]) < TOL:
            active_rows.discard(i)
        if abs(demand[j]) < TOL:
            active_cols.discard(j)
    return plan, cost, added

def _two_smallest(values: List[float]) -> Tuple[float, float]:
    inf = float("inf"); a, b = inf, inf
    for v in values:
        if v < a: a, b = v, a
        elif v < b: b = v
    return a, b

def vogel_approximation(supply: List[float], demand: List[float], cost: List[List[float]], verbose=True):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0 for _ in range(n)] for _ in range(m)]
    active_rows = {i for i in range(m) if supply[i] > 0}
    active_cols = {j for j in range(n) if demand[j] > 0}
    if verbose:
        print("=== Vogel's Approximation Method ===")
        print("Balanced problem:")
        print(f"  supply = {supply}")
        print(f"  demand = {demand}")
        print(f"  added = {added}\n")
    step = 0
    while active_rows and active_cols:
        step += 1
        # penalties
        rpen, cpen = {}, {}
        for i in active_rows:
            vals = [cost[i][j] for j in active_cols]
            m1, m2 = _two_smallest(vals); rpen[i] = (m1, m2, (m2 - m1) if m2 != float("inf") else m1)
        for j in active_cols:
            vals = [cost[i][j] for i in active_rows]
            m1, m2 = _two_smallest(vals); cpen[j] = (m1, m2, (m2 - m1) if m2 != float("inf") else m1)
        # choose by max penalty
        best = None
        chosen_type, chosen_idx = None, None
        for i in active_rows:
            m1,m2,p = rpen[i]; score = (p, -m1, 1, -i)
            if best is None or score > best: best, chosen_type, chosen_idx = score, 'row', i
        for j in active_cols:
            m1,m2,p = cpen[j]; score = (p, -m1, 0, -j)
            if best is None or score > best: best, chosen_type, chosen_idx = score, 'col', j
        if chosen_type == 'row':
            i = chosen_idx
            j = min(active_cols, key=lambda jj: (cost[i][jj], jj))
        else:
            j = chosen_idx
            i = min(active_rows, key=lambda ii: (cost[ii][j], ii))
        alloc = min(supply[i], demand[j])
        plan[i][j] += alloc
        supply[i] -= alloc; demand[j] -= alloc
        print(f"Step {step}: Allocate {alloc} units to cell ({i},{j}) with cost={cost[i][j]}")
        print(f"  Remaining supply[{i}]={supply[i]}, demand[{j}]={demand[j]}\n")
        if abs(supply[i]) < TOL: active_rows.discard(i)
        if abs(demand[j]) < TOL: active_cols.discard(j)
    return plan, cost, added

# -------------------- Solve Function --------------------
def solve_transport_jupyter(supply, demand, cost, method='vam', do_optimal=False):
    method = method.lower()
    if method == 'nwc':
        plan, cost_mat, added = northwest_corner(supply[:], demand[:], cost)
    elif method == 'lcm':
        plan, cost_mat, added = least_cost_method(supply[:], demand[:], cost)
    elif method == 'vam':
        plan, cost_mat, added = vogel_approximation(supply[:], demand[:], cost)
    else:
        raise ValueError("Method must be one of: nwc, lcm, vam")

    print_transport_table(plan, cost_mat, title=f"{method.upper()} Initial Feasible Plan")
    print(f"Initial total cost = {total_cost(plan, cost_mat):.4f}")
    if added["dummy_row"] or added["dummy_col"]:
        print("Note: Dummy row/column was added for balancing (cost=0).")

# -------------------- Example: Run in Jupyter --------------------
# Choose method: 'nwc', 'lcm', 'vam'
solve_transport_jupyter(SUPPLY, DEMAND, COST, method='vam')


=== Vogel's Approximation Method ===
Balanced problem:
  supply = [400, 500]
  demand = [300, 200, 400]
  added = {'dummy_row': False, 'dummy_col': False}

Step 1: Allocate 300 units to cell (1,0) with cost=400
  Remaining supply[1]=200, demand[0]=0

Step 2: Allocate 200 units to cell (1,2) with cost=600
  Remaining supply[1]=0, demand[2]=200

Step 3: Allocate 200 units to cell (0,1) with cost=800
  Remaining supply[0]=200, demand[1]=0

Step 4: Allocate 200 units to cell (0,2) with cost=700
  Remaining supply[0]=0, demand[2]=0


VAM Initial Feasible Plan
---------------------------------------------------------------------------------
Src/Dest           |       Dest1        |       Dest2        |       Dest3       
---------------------------------------------------------------------------------
Src1               |                    |  200.0000@800.00   |  200.0000@700.00  
Src2               |  300.0000@400.00   |                    |  200.0000@600.00  
-----------------------------

In [ ]:
# -*- coding: utf-8 -*-
"""
Transport Suite for Jupyter Notebook:
NWC / LCM / VAM + Stepping-Stone
Interactive, step-by-step solution.
"""

from typing import List, Tuple, Set
import collections

TOL = 1e-12

# -------------------- Edit your data here --------------------
SUPPLY = [400, 500]
DEMAND = [300, 200, 400]
COST = [
    [600, 800, 700],
    [400, 900, 600],
]

# If you want to maximize, negate costs:
COST_NEG = [[-p for p in row] for row in COST]
# -------------------------------------------------------------

# -------------------- Utilities --------------------
def balance_transport(supply: List[float], demand: List[float], cost: List[List[float]]):
    S, D = sum(supply), sum(demand)
    m, n = len(supply), len(demand)
    C = [row[:] for row in cost]
    added = {"dummy_row": False, "dummy_col": False}
    if abs(S - D) < 1e-9:
        return supply[:], demand[:], C, added
    if S < D:
        C.append([0.0] * n)
        supply = supply[:] + [D - S]
        added["dummy_row"] = True
    else:
        for r in C:
            r.append(0.0)
        demand = demand[:] + [S - D]
        added["dummy_col"] = True
    return supply, demand, C, added

def total_cost(plan, cost) -> float:
    return sum(plan[i][j] * cost[i][j] for i in range(len(plan)) for j in range(len(plan[0])))

def print_transport_table(plan: List[List[float]], cost: List[List[float]], title="Plan (qty@unit_cost)"):
    m, n = len(plan), len(plan[0])
    colw = 18
    sep = " | "
    print("\n" + title)
    print("-" * (colw * (n + 1) + len(sep) * n))
    header = ["Src/Dest".ljust(colw)] + [f"Dest{j+1}".center(colw) for j in range(n)]
    print(sep.join(header))
    print("-" * (colw * (n + 1) + len(sep) * n))
    for i in range(m):
        row = [f"Src{i+1}".ljust(colw)]
        for j in range(n):
            cell = f"{plan[i][j]:.4f}@{cost[i][j]:.2f}" if plan[i][j] != 0 else "".center(colw)
            row.append(cell.center(colw))
        print(sep.join(row))
    print("-" * (colw * (n + 1) + len(sep) * n))

# -------------------- Initial Methods --------------------
def northwest_corner(supply, demand, cost):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0]*n for _ in range(m)]
    i = j = 0
    while i < m and j < n:
        alloc = min(supply[i], demand[j])
        plan[i][j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc
        if abs(supply[i]) < TOL and abs(demand[j]) < TOL:
            i += 1; j += 1
        elif abs(supply[i]) < TOL:
            i += 1
        elif abs(demand[j]) < TOL:
            j += 1
    return plan, cost, added

def least_cost_method(supply, demand, cost):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0]*n for _ in range(m)]
    active_rows = {i for i in range(m) if supply[i] > 0}
    active_cols = {j for j in range(n) if demand[j] > 0}
    while active_rows and active_cols:
        best = None
        for i in active_rows:
            for j in active_cols:
                cand = (cost[i][j], i, j)
                if best is None or cand < best: best = cand
        cmin, i, j = best
        alloc = min(supply[i], demand[j])
        plan[i][j] += alloc
        supply[i] -= alloc
        demand[j] -= alloc
        if abs(supply[i]) < TOL: active_rows.discard(i)
        if abs(demand[j]) < TOL: active_cols.discard(j)
    return plan, cost, added

def vogel_approximation(supply, demand, cost):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0]*n for _ in range(m)]
    active_rows = {i for i in range(m) if supply[i] > 0}
    active_cols = {j for j in range(n) if demand[j] > 0}

    def _two_smallest(values):
        inf = float("inf"); a,b = inf,inf
        for v in values:
            if v < a: a,b=v,a
            elif v < b: b=v
        return a,b

    step = 0
    while active_rows and active_cols:
        step += 1
        rpen,cpen = {},{}
        for i in active_rows:
            vals = [cost[i][j] for j in active_cols]
            m1,m2 = _two_smallest(vals); rpen[i]=(m1,m2,(m2-m1 if m2!=float('inf') else m1))
        for j in active_cols:
            vals = [cost[i][j] for i in active_rows]
            m1,m2 = _two_smallest(vals); cpen[j]=(m1,m2,(m2-m1 if m2!=float('inf') else m1))
        best = None; chosen_type=None; chosen_idx=None
        for i in active_rows:
            m1,m2,p = rpen[i]; score = (p, -m1, 1, -i)
            if best is None or score>best: best,chosen_type,chosen_idx = score,'row',i
        for j in active_cols:
            m1,m2,p = cpen[j]; score = (p,-m1,0,-j)
            if best is None or score>best: best,chosen_type,chosen_idx = score,'col',j
        if chosen_type=='row':
            i=chosen_idx; j=min(active_cols,key=lambda jj:(cost[i][jj],jj))
        else:
            j=chosen_idx; i=min(active_rows,key=lambda ii:(cost[ii][j],ii))
        alloc = min(supply[i],demand[j])
        plan[i][j] += alloc
        supply[i] -= alloc; demand[j] -= alloc
        print(f"Step {step}: Allocate {alloc} units to cell ({i},{j}) with cost={cost[i][j]}")
        print(f"  Remaining supply[{i}]={supply[i]}, demand[{j}]={demand[j]}\n")
        if abs(supply[i])<TOL: active_rows.discard(i)
        if abs(demand[j])<TOL: active_cols.discard(j)
    return plan,cost,added

# -------------------- Stepping-Stone Improvement --------------------
def positives_as_basics(plan):
    basics = set()
    for i,row in enumerate(plan):
        for j,x in enumerate(row):
            if x > TOL: basics.add((i,j))
    return basics

class DSU:
    def __init__(self,n): self.p=list(range(n))
    def find(self,x):
        while self.p[x]!=x: self.p[x]=self.p[self.p[x]]; x=self.p[x]
        return x
    def union(self,a,b):
        ra,rb=self.find(a),self.find(b)
        if ra!=rb: self.p[rb]=ra; return True
        return False

def ensure_spanning_tree(plan,cost,basic_zeros:Set[Tuple[int,int]]):
    m,n=len(plan),len(plan[0])
    N=m+n
    dsu=DSU(N)
    basics=positives_as_basics(plan).union(basic_zeros)
    edges=[]
    for (i,j) in basics:
        if dsu.union(i,m+j): edges.append((i,j))
    while len(edges)<m+n-1:
        candidate=None; best_cost=float('inf')
        for i in range(m):
            for j in range(n):
                if (i,j) in basics: continue
                if dsu.find(i)!=dsu.find(m+j):
                    c=cost[i][j]
                    if c<best_cost: best_cost, candidate=c,(i,j)
        if candidate is None: break
        i,j=candidate
        basic_zeros.add((i,j)); basics.add((i,j)); dsu.union(i,m+j); edges.append((i,j))
    return basic_zeros

def build_basis_graph(plan,basic_zeros):
    m,n=len(plan),len(plan[0])
    basics=positives_as_basics(plan).union(basic_zeros)
    g=collections.defaultdict(list)
    for (i,j) in basics:
        g[('r',i)].append(('c',j)); g[('c',j)].append(('r',i))
    return g

def path_row_to_col(g,i0,j0):
    from collections import deque
    start=('r',i0); goal=('c',j0)
    q=deque([start]); prev={start:None}
    while q:
        u=q.popleft()
        if u==goal: break
        for v in g[u]:
            if v not in prev: prev[v]=u; q.append(v)
    if goal not in prev: return None
    nodes=[]; cur=goal
    while cur is not None: nodes.append(cur); cur=prev[cur]
    nodes.reverse()
    cells=[]
    for a,b in zip(nodes[:-1],nodes[1:]):
        if a[0]=='r' and b[0]=='c': cells.append((a[1],b[1]))
        elif a[0]=='c' and b[0]=='r': cells.append((b[1],a[1]))
        else: raise RuntimeError("Invalid alternation")
    return cells

def loop_for_cell(plan,basic_zeros,start):
    i0,j0=start
    g=build_basis_graph(plan,basic_zeros)
    path_cells=path_row_to_col(g,i0,j0)
    if not path_cells: return None
    return [start]+path_cells+[start]

def compute_delta(cost,loop):
    delta=0.0
    for k in range(len(loop)-1):
        i,j=loop[k]
        delta+=cost[i][j]*(1 if k%2==0 else -1)
    return delta

def stepping_stone(plan,cost):
    m,n=len(plan),len(plan[0])
    basic_zeros:Set[Tuple[int,int]]=set()
    basic_zeros=ensure_spanning_tree(plan,cost,basic_zeros)
    iter_no=0
    while True:
        iter_no+=1
        best=None
        basics_now=positives_as_basics(plan).union(basic_zeros)
        print(f"\n=== Iteration {iter_no} ===")
        for i in range(m):
            for j in range(n):
                if (i,j) in basics_now: continue
                loop=loop_for_cell(plan,basic_zeros,(i,j))
                if not loop: continue
                delta=compute_delta(cost,loop)
                print(f"  cell({i},{j}) -> Δ={delta:.4f} loop={loop}")
                if best is None or delta<best[0]: best=(delta,loop)
        if best is None or best[0]>=-1e-12:
            print("No negative Δ -> current plan is optimal.")
            break
        delta,loop=best
        minus_cells=[loop[k] for k in range(1,len(loop)-1,2)]
        theta=min(plan[i][j] for (i,j) in minus_cells)
        print(f"\nEntering cell {loop[0]} with most negative Δ={delta:.4f}")
        print(f" '-' positions: {minus_cells} -> θ={theta:.4f}")
        # Update allocations along the loop
        for k in range(len(loop)-1):
            i,j = loop[k]
            if k % 2 == 0:
                plan[i][j] += theta
            else:
                plan[i][j] -= theta
                if plan[i][j] < TOL:
                    plan[i][j] = 0.0
        # Refresh zero-basics and ensure spanning tree
        basic_zeros = {c for c in basic_zeros if c not in minus_cells}
        basic_zeros = ensure_spanning_tree(plan, cost, basic_zeros)
        print(f"Updated total cost = {total_cost(plan, cost):.4f}")
        print_transport_table(plan, cost, title="Updated Plan After Iteration")
    return plan



In [11]:
# Choose initial method: 'nwc', 'lcm', or 'vam'
initial_method = 'vam'

if initial_method == 'nwc':
    plan, cost_mat, added = northwest_corner(SUPPLY, DEMAND, COST)
elif initial_method == 'lcm':
    plan, cost_mat, added = least_cost_method(SUPPLY, DEMAND, COST)
elif initial_method == 'vam':
    plan, cost_mat, added = vogel_approximation(SUPPLY, DEMAND, COST)
else:
    raise ValueError("Method must be 'nwc', 'lcm', or 'vam'")

print_transport_table(plan, cost_mat, title=f"{initial_method.upper()} Initial Feasible Plan")
print(f"Initial total cost = {total_cost(plan, cost_mat):.4f}")

# Step-by-step Stepping-Stone improvement
optimal_plan = stepping_stone(plan, cost_mat)
print("\n=== Optimal Plan ===")
print_transport_table(optimal_plan, cost_mat)
print(f"Optimal total cost = {total_cost(optimal_plan, cost_mat):.4f}")


=== Vogel's Approximation Method ===
Balanced problem:
  supply = [400, 500]
  demand = [300, 200, 400]
  added = {'dummy_row': False, 'dummy_col': False}

Step 1: Allocate 300 units to cell (1,0) with cost=400
  Remaining supply[1]=200, demand[0]=0

Step 2: Allocate 200 units to cell (1,2) with cost=600
  Remaining supply[1]=0, demand[2]=200

Step 3: Allocate 200 units to cell (0,1) with cost=800
  Remaining supply[0]=200, demand[1]=0

Step 4: Allocate 200 units to cell (0,2) with cost=700
  Remaining supply[0]=0, demand[2]=0


VAM Initial Feasible Plan
---------------------------------------------------------------------------------
Src/Dest           |       Dest1        |       Dest2        |       Dest3       
---------------------------------------------------------------------------------
Src1               |                    |  200.0000@800.00   |  200.0000@700.00  
Src2               |  300.0000@400.00   |                    |  200.0000@600.00  
-----------------------------

In [13]:
def vogel_approximation_verbose(supply, demand, cost):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0 for _ in range(n)] for _ in range(m)]
    active_rows = {i for i in range(m) if supply[i] > 0}
    active_cols = {j for j in range(n) if demand[j] > 0}

    def _two_smallest(values):
        inf = float("inf"); a,b = inf,inf
        for v in values:
            if v < a: a,b=v,a
            elif v < b: b=v
        return a,b

    step = 0
    while active_rows and active_cols:
        step += 1
        # Calculate row and column penalties
        rpen, cpen = {}, {}
        for i in active_rows:
            vals = [cost[i][j] for j in active_cols]
            m1,m2 = _two_smallest(vals)
            rpen[i] = (m1,m2,(m2-m1 if m2!=float('inf') else m1))
        for j in active_cols:
            vals = [cost[i][j] for i in active_rows]
            m1,m2 = _two_smallest(vals)
            cpen[j] = (m1,m2,(m2-m1 if m2!=float('inf') else m1))
        
        # Choose max penalty
        best = None; chosen_type=None; chosen_idx=None
        for i in active_rows:
            m1,m2,p = rpen[i]; score = (p, -m1, 1, -i)
            if best is None or score>best: best,chosen_type,chosen_idx = score,'row',i
        for j in active_cols:
            m1,m2,p = cpen[j]; score = (p,-m1,0,-j)
            if best is None or score>best: best,chosen_type,chosen_idx = score,'col',j

        # Find allocation cell
        if chosen_type=='row':
            i=chosen_idx
            j=min(active_cols,key=lambda jj:(cost[i][jj],jj))
        else:
            j=chosen_idx
            i=min(active_rows,key=lambda ii:(cost[ii][j],ii))
        
        alloc = min(supply[i],demand[j])
        plan[i][j] += alloc
        supply[i] -= alloc; demand[j] -= alloc
        print(f"Step {step}: Allocate {alloc:.4f} to cell ({i},{j}) with cost={cost[i][j]:.2f}")
        print(f"  Remaining supply[{i}]={supply[i]:.4f}, demand[{j}]={demand[j]:.4f}\n")
        if abs(supply[i])<TOL: active_rows.discard(i)
        if abs(demand[j])<TOL: active_cols.discard(j)
    return plan,cost,added

def least_cost_method_verbose(supply, demand, cost):
    supply, demand, cost, added = balance_transport(supply, demand, cost)
    m, n = len(supply), len(demand)
    plan = [[0.0 for _ in range(n)] for _ in range(m)]
    active_rows = {i for i in range(m) if supply[i] > 0}
    active_cols = {j for j in range(n) if demand[j] > 0}
    step = 0
    while active_rows and active_cols:
        step += 1
        # Find cell with minimum cost
        best = None
        for i in active_rows:
            for j in active_cols:
                cand = (cost[i][j], i, j)
                if best is None or cand < best:
                    best = cand
        cmin, i, j = best
        alloc = min(supply[i], demand[j])
        plan[i][j] += alloc
        supply[i] -= alloc
        demand[j] -= alloc
        print(f"Step {step}: allocate {alloc:.4f} to cell ({i},{j}) with cost={cmin:.2f}")
        print(f"  Remaining supply[{i}]={supply[i]:.4f}, demand[{j}]={demand[j]:.4f}\n")
        if abs(supply[i]) < TOL: active_rows.discard(i)
        if abs(demand[j]) < TOL: active_cols.discard(j)
    return plan, cost, added


In [14]:
# Step-by-step Least Cost Method
plan_lcm, cost_mat, added = least_cost_method_verbose(SUPPLY, DEMAND, COST)
print_transport_table(plan_lcm, cost_mat, title="LCM Initial Feasible Plan")
print(f"Total cost = {total_cost(plan_lcm, cost_mat):.4f}")

# Step-by-step Vogel Approximation Method
plan_vam, cost_mat, added = vogel_approximation_verbose(SUPPLY, DEMAND, COST)
print_transport_table(plan_vam, cost_mat, title="VAM Initial Feasible Plan")
print(f"Total cost = {total_cost(plan_vam, cost_mat):.4f}")


Step 1: allocate 300.0000 to cell (1,0) with cost=400.00
  Remaining supply[1]=200.0000, demand[0]=0.0000

Step 2: allocate 200.0000 to cell (1,2) with cost=600.00
  Remaining supply[1]=0.0000, demand[2]=200.0000

Step 3: allocate 200.0000 to cell (0,2) with cost=700.00
  Remaining supply[0]=200.0000, demand[2]=0.0000

Step 4: allocate 200.0000 to cell (0,1) with cost=800.00
  Remaining supply[0]=0.0000, demand[1]=0.0000


LCM Initial Feasible Plan
---------------------------------------------------------------------------------
Src/Dest           |       Dest1        |       Dest2        |       Dest3       
---------------------------------------------------------------------------------
Src1               |                    |  200.0000@800.00   |  200.0000@700.00  
Src2               |  300.0000@400.00   |                    |  200.0000@600.00  
---------------------------------------------------------------------------------
Total cost = 540000.0000
Step 1: Allocate 300.0000 to c